[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/HyperCoast/blob/main/docs/workshops/pace.ipynb)

# Working with NASA PACE data in HyperCoast

This notebook demonstrates how to visualize and analyze Plankton, Aerosol, Cloud, ocean Ecosystem ([PACE](https://pace.gsfc.nasa.gov/)) data interactively with HyperCoast.

## Environment setup

Uncomment and run the following cell to install the required packages.

In [ ]:
# %pip install "hypercoast[extra]"

Import libraries.

In [ ]:
import earthaccess
import hypercoast
import pandas as pd

## Search for PACE data

To download and access the data, you will need to create an Earthdata login. You can register for an account at [urs.earthdata.nasa.gov](https://urs.earthdata.nasa.gov). Once you have an account, run the following cell and enter your NASA Earthdata login credentials.

In [ ]:
earthaccess.login(persist=True)

### Search data programmatically

To search for PACE data programmatically, specify the bounding box and time range of interest. Set `count=-1` to return all results or set `count=10` to return the first 10 results.

In [ ]:
results, gdf = hypercoast.search_pace(
    bounding_box=(-83, 25, -81, 28),
    temporal=("2024-07-30", "2024-08-15"),
    short_name="PACE_OCI_L2_AOP_NRT",
    count=10,
    return_gdf=True,
)

Plot the footprints of the returned datasets on a map.

In [ ]:
gdf.explore()

Download the first dataset from the search results. Note that the download may take some time.

In [ ]:
hypercoast.download_pace(results[:1], out_dir="data")

### Search data interactively

To search for PACE data interactively, pan and zoom to the area of interest. Specify the time range of interest from the search dialog, then click on the Search button.

In [ ]:
m = hypercoast.Map(center=[30.0262, -90.1345], zoom=8)
m.search_pace(default_dataset="PACE_OCI_L2_AOP_NRT")
m

By default, the `search_pace` method searches for the `PACE_OCI_L2_AOP_NRT` dataset, but you can specify the dataset name by setting the `default_dataset` parameter, such as `PACE_OCI_L2_BGC_NRT`. For more information about the available datasets, see the [PACE Data Products](https://pace.oceansciences.org/data_table.htm) page.

![image](https://github.com/user-attachments/assets/2c45ad43-c405-402a-92e8-42f497730fbb)

Uncomment the following cell to display the GeoDataFrame of the search results.

In [ ]:
# m._NASA_DATA_GDF.head()

Similarly, you can download the first dataset from the search results by uncommenting the following cell.

In [ ]:
# hypercoast.download_pace(results[:1], out_dir="data")

## Read PACE data

Let's download a sample PACE Apparent Optical Properties ([AOP](https://pace.oceansciences.org/pace_eq_aop.htm)) dataset for the demonstration.

In [ ]:
results = hypercoast.search_pace(
    bounding_box=(-83, 25, -81, 28),
    temporal=("2024-07-30", "2024-08-15"),
    short_name="PACE_OCI_L2_AOP_NRT",
    count=1,
)

In [ ]:
hypercoast.download_pace(results[:1], out_dir="data")

Let's make a scatter plot of the pixel locations so we can see the irregular spacing.

In [ ]:
filepath = "data/PACE_OCI.20240730T181157.L2.OC_AOP.V2_0.NRT.nc"
plot = hypercoast.view_pace_pixel_locations(filepath, step=20)

Load the dataset as a `xarray.Dataset` object.

In [ ]:
dataset = hypercoast.read_pace(filepath)
# dataset

![image](https://github.com/user-attachments/assets/56b1fae3-9acf-4ee1-8dc9-7f6784bedf88)

## Visualize PACE AOP data

Visualize selected bands of the dataset.

In [ ]:
hypercoast.viz_pace(dataset, wavelengths=[500, 510, 520, 530], ncols=2)

Add custom projection and administrative boundaries to the map. The default projection is `PlateCarree`. You can specify a custom projection by setting the `crs` parameter. For more information about the available projections, see the [cartopy projection](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html) page.

In [ ]:
hypercoast.viz_pace(dataset, wavelengths=[500, 510, 520, 530], ncols=2, crs="default")

## Plot spectral signatures

Plot the spectral signature of a pixel using the `extract_pace` function. Set `return_plot=True` to return the plot object. 

In [ ]:
latitude = 29.9307
longitude = -87.9106
hypercoast.extract_pace(dataset, latitude, longitude, return_plot=True)

To return the extracted values as an xarray `DataArray`, set `return_plot=False`.

In [ ]:
array = hypercoast.extract_pace(dataset, latitude, longitude, return_plot=False)
# array

To plot the spectral signatures of multiple pixels, you can specify the pixel locations as a list of tuples. All pixels within the specified latitude and longitude range will be extracted.

In [ ]:
latitude = (29.49, 29.50)
longitude = (-88.10, -88.00)
hypercoast.filter_pace(dataset, latitude, longitude, return_plot=True)

## Interactive visualization

### Single-band visualization

Visualize a selected band of the dataset interactively use the `add_pace` method and speccify the `wavelengths` parameter.

In [ ]:
m = hypercoast.Map()
m.add_basemap("Hybrid")
wavelengths = [450]
m.add_pace(dataset, wavelengths, colormap="jet", vmin=0, vmax=0.02, layer_name="PACE")
m.add_colormap(cmap="jet", vmin=0, vmax=0.02, label="Reflectance")
m.add("spectral")
m.set_center(-80.7382, 26.5295, zoom=6)
m

Click on the map to display the spectral signature of the selected pixel. 

![image](https://github.com/user-attachments/assets/6c8fd406-4655-4e2e-9270-ae3c5f484121)

Convert the spectral data of the selected pixels to a DataFrame.

In [ ]:
df = m.spectral_to_df()
df.head()

Convert the spectral data of the selected pixels to a GeoDataFrame.

In [ ]:
gdf = m.spectral_to_gdf()
gdf.head()

Convert the spectral data of the selected pixels to a CSV file.

In [ ]:
m.spectral_to_csv("data/spectral.csv")

### Multi-band visualization

Select three spectral bands to visualize as an RGB image.

In [ ]:
m = hypercoast.Map()
m.add_basemap("Hybrid")
wavelengths = [450, 550, 650]
m.add_pace(
    dataset, wavelengths, indexes=[3, 2, 1], vmin=0, vmax=0.02, layer_name="PACE"
)
m.add("spectral")
m.set_center(-80.7382, 26.5295, zoom=6)
m

![image](https://github.com/user-attachments/assets/a5b28084-f958-437c-b492-376a15451baa)

### Change band combination

Click on the gear icon on the toolbar to change the band combination.

![image](https://github.com/user-attachments/assets/2b50d6cb-92df-4fdb-a56d-426d3bca2777)


## PACE BGC data

PACE has a variety of data products, including biogeochemical properties. For more information about the available datasets, see the [PACE Data Products](https://pace.oceansciences.org/data_table.htm) page.

The PACE Biogeochemical (BGC) data products include chlorophyll-a concentration, particulate organic carbon, and particulate inorganic carbon.

### Download PACE BGC data

Let's download a sample PACE BGC dataset for the demonstration.

In [ ]:
results, gdf = hypercoast.search_nasa_data(
    short_name="PACE_OCI_L2_BGC_NRT",
    bbox=(-90.5642, 29.9749, -89.7143, 30.42),
    temporal=("2024-07-30", "2024-08-15"),
    count=1,
    return_gdf=True,
)
hypercoast.download_nasa_data(results, out_dir="data")

Load the downloaded dataset as an `xarray.Dataset`:

In [ ]:
filepath = "data/PACE_OCI.20240730T181157.L2.OC_BGC.V2_0.NRT.nc"
dataset = hypercoast.read_pace_bgc(filepath)

Let's inspect the data variables contained in the dataset:

In [ ]:
dataset.variables

We can see that the dataset contains the following variables:
- [Chlorophyll Concentration](https://www.earthdata.nasa.gov/apt/documents/chlor-a/v1.0)
- [Phytoplankton Carbon](https://oceancolor.gsfc.nasa.gov/resources/atbd/cphyt/)
- [Particulate Organic Carbon](https://oceancolor.gsfc.nasa.gov/resources/atbd/poc/)

### Visualize PACE BGC data

Since the datasets are not gridded, we need to transform them into gridded data to visualize them. We can use the `grid_pace_bgc` function to transform the dataset into a gridded format.

First, transform the `chlor_a` variable into a gridded format:

In [ ]:
chlor_a = hypercoast.grid_pace_bgc(dataset, variable="chlor_a", method="linear")

Plot the gridded Chlorophyll Concentration data:

In [ ]:
chlor_a.plot(vmin=0, vmax=20, cmap="jet", size=6)

Plot the gridded Phytoplankton Carbon data:

In [ ]:
carbon_phyto = hypercoast.grid_pace_bgc(
    dataset, variable="carbon_phyto", method="linear"
)
carbon_phyto.plot(vmin=0, vmax=120, cmap="jet", size=6)

Plot the gridded Particulate Organic Carbon data:

In [ ]:
poc = hypercoast.grid_pace_bgc(dataset, variable="poc", method="linear")
poc.plot(vmin=0, vmax=1000, cmap="jet")

Plot the gridded BGC data on an interactive map.

In [ ]:
m = hypercoast.Map()
m.add_basemap("Hybrid")
m.add_raster(chlor_a, layer_name="Chlorophyll-a", colormap="jet", vmin=0, vmax=20)
m.add_raster(
    carbon_phyto, layer_name="Phytoplankton Carbon", colormap="plasma", vmin=0, vmax=120
)
m.add_raster(
    poc, layer_name="Particulate Organic Carbon", colormap="coolwarm", vmin=0, vmax=1000
)
m.add_layer_manager()

m.add_colormap(cmap="jet", vmin=0, vmax=20, label="Chlorophyll-a (mg/m3)")
m.add_colormap(cmap="plasma", vmin=0, vmax=120, label="Phytoplankton Carbon (mg/m3)")
m.add_colormap(
    cmap="coolwarm", vmin=0, vmax=1000, label="Particulate Organic Carbon (mg/m3)"
)
m.set_center(-80.7382, 26.5295, zoom=6)
m

![](https://i.imgur.com/AEccc5k.gif)

## PACE Chlorophyll Level 3 data

PACE Level 3 data products are gridded data products that are derived from Level 2 data. Once of the most common Level 3 data products is the Chlorophyll-Carotenoid Index (CCI) dataset.

Let's download some daily PACE Chlorophyll Level 3 data for the demonstration.

In [ ]:
temporal = ("2024-07-30", "2024-08-15")
results = hypercoast.search_pace_chla(temporal=temporal)
hypercoast.download_nasa_data(results, "chla")

The downloaded datasets can be found in the `chla` directory, which contains 17 daily files of CCI data in the netCDF format. The date range of the data is from 2024-07-30 to 2024-08-15.

In [ ]:
files = "chla/*nc"

Load all the data files in the `chla` directory as an xarray DataArray

In [ ]:
array = hypercoast.read_pace_chla(files)
# array

![image](https://github.com/user-attachments/assets/3a0011dd-a16a-49e9-8f49-7d96aef1ce1f)

Select a date and visualize the chlorophyll-a concentration data with Matplotlib.

In [ ]:
hypercoast.viz_pace_chla(array, date="2024-07-30", cmap="jet", size=6)

If the date is not specified, the data are averaged over the entire time range.

In [ ]:
hypercoast.viz_pace_chla(array, cmap="jet", size=6)

To visualize the data interactively, we can select either a single date or aggregate the data over a time range. 

First, let's select a single date from the data array:

In [ ]:
single_array = array.sel(date="2024-07-30")
# single_array

Convert the data array to an image that can be displayed on an interactive map.

In [ ]:
single_image = hypercoast.pace_chla_to_image(single_array)

Create an interactive map and display the image on the map.

In [ ]:
m = hypercoast.Map(center=[40, -100], zoom=4)
m.add_basemap("Hybrid")
m.add_raster(
    single_image,
    cmap="jet",
    vmin=-1,
    vmax=2,
    layer_name="Chlorophyll a",
    zoom_to_layer=False,
)
label = "Chlorophyll Concentration [lg(lg(mg m^-3))]"
m.add_colormap(cmap="jet", vmin=-1, vmax=2, label=label)
m

![image](https://github.com/user-attachments/assets/3d6e41ac-3a42-4296-a459-93ad7509b379)

The daily image does not have a global coverage. To visualize the data globally, we can aggregate the data over a time range.

In [ ]:
mean_array = array.mean(dim="date")

Convert the aggregated data array to an image that can be displayed on an interactive map.

In [ ]:
image = hypercoast.pace_chla_to_image(mean_array)

Create an interactive map and display the image on the map.

In [ ]:
m = hypercoast.Map(center=[40, -100], zoom=4)
m.add_basemap("Hybrid")
m.add_raster(
    image, cmap="jet", vmin=-1, vmax=2, layer_name="Chlorophyll a", zoom_to_layer=False
)
label = "Chlorophyll Concentration [lg(lg(mg m^-3))]"
m.add_colormap(cmap="jet", vmin=-1, vmax=2, label=label)
m

![image](https://github.com/user-attachments/assets/f33cf980-e528-4fd9-bdf7-b1a45b6ca5d4)

## Hypoxia Cruise data

The [Hypoxia Cruise](https://www.noaa.gov/media-advisory/noaa-partners-to-report-on-2024-gulf-of-mexico-dead-zone-monitoring-cruise) collected water quality data in the Gulf of Mexico from July 21 to August 2, 2024. In this section, we will visualize the cruise sampling locations.

First, let's download an Excel file containing the cruise sampling locations.

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/hypercoast/Hypoxia_Data_Sheet.xlsx"
xls_path = "data/Hypoxia_Data_Sheet.xlsx"
hypercoast.download_file(url, xls_path, overwrite=True)

In [ ]:
df = pd.read_excel(xls_path)
df.head()

Filter the data to select only the sampling locations with latitude and longitude coordinates.

In [ ]:
df_filtered = df.dropna(subset=["Lon", "Lat"]).reset_index(drop=True)
df_filtered.head()

Download the KML file containing the cruise path.

In [ ]:
url = (
    "https://github.com/opengeos/datasets/releases/download/hypercoast/Hypoxia_Path.kml"
)
kml_path = "data/Hypoxia_Path.kml"
hypercoast.download_file(url, kml_path)

We will use the PACE AOP dataset acquired on July 30, 2024, to visualize the cruise sampling locations. The dataset should have been downloaded in the previous section.

In [ ]:
filepath = "data/PACE_OCI.20240730T181157.L2.OC_AOP.V2_0.NRT.nc"

Read the PACE AOP dataset as an xarray Dataset.

In [ ]:
dataset = hypercoast.read_pace(filepath)
# dataset

Visualize the cruise sampling locations and PACE data on the map.

In [ ]:
m = hypercoast.Map()
m.add_basemap("Hybrid")
wavelengths = [450, 550, 650]
m.add_pace(
    dataset, wavelengths, indexes=[3, 2, 1], vmin=0, vmax=0.02, layer_name="PACE"
)
m.add("spectral")
style = {"weight": 2, "color": "red"}
m.add_kml(kml_path, style=style, layer_name="Hypoxia Path", info_mode=None)
m.add_points_from_xy(
    df_filtered,
    x="Lon",
    y="Lat",
    max_cluster_radius=50,
    layer_name="Hypoxia Data Points",
)
m.set_center(-91.46118, 28.89758, zoom=8)
m

![image](https://github.com/user-attachments/assets/9a4bc3b7-2a69-4a94-a4f8-297103cb80d3)

## Visualize in-situ data

This section demonstrates how to visualize in-situ data on the map. First, let's download a hypothetical in-situ dataset.

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/hypercoast/pace_sample_points.csv"
data = pd.read_csv(url)
data.head()

Again, we will use the PACE AOP dataset acquired on July 30, 2024, to visualize the in-situ data. The dataset should have been downloaded in the previous section.

In [ ]:
filepath = "data/PACE_OCI.20240730T181157.L2.OC_AOP.V2_0.NRT.nc"

Read the PACE dataset as an xarray Dataset.

In [ ]:
dataset = hypercoast.read_pace(filepath)

Visualize the in-situ data on the map.

In [ ]:
m = hypercoast.Map(center=[27.235094, -87.791748], zoom=6)

m.add_basemap("Hybrid")
wavelengths = [450]
m.add_pace(dataset, wavelengths, colormap="jet", vmin=0, vmax=0.02, layer_name="PACE")
m.add_colormap(cmap="jet", vmin=0, vmax=0.02, label="Reflectance")
m.add("spectral")

m.add_field_data(
    data,
    x_col="wavelength",
    y_col_prefix="(",
    x_label="Wavelength (nm)",
    y_label="Reflectance",
    use_marker_cluster=True,
)
m.set_center(-87.791748, 27.235094, zoom=6)
m

Click on any marker to display the in-situ data.

![image](https://github.com/user-attachments/assets/f4ccaa34-c7d1-47cb-97e5-9c36a86272c8)

## Analyze PACE data

To anyalyze the PACE data with algorithms, such as K-means clustering, principal component analysis (PCA), or Spectral Angle Mapper (SAM), follow the notebook at https://hypercoast.org/examples/pace_cyano.